In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Ensure that the date column is datetime and data is sorted
trunc_input['ds'] = pd.to_datetime(trunc_input['ds'])
trunc_input = trunc_input.sort_values('ds').reset_index(drop=True)

# Scale the target values to help the network learn
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_y = scaler.fit_transform(trunc_input[['y']])

# Function to create sequences for LSTM training
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

sequence_length = 12  # using past 12 periods to predict the next period
X, y = create_sequences(scaled_y, sequence_length)
# Reshape X to be [samples, timesteps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build a simple LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=50, batch_size=16, verbose=1)

# Forecasting: use the last available sequence to predict 12 future steps iteratively
last_sequence = scaled_y[-sequence_length:]  # last sequence from the training data
forecast_scaled = []
current_sequence = last_sequence.copy()

for _ in range(12):
    # Reshape current sequence for prediction: (1, sequence_length, 1)
    current_sequence_reshaped = np.reshape(current_sequence, (1, sequence_length, 1))
    # Predict next value
    pred = model.predict(current_sequence_reshaped)
    forecast_scaled.append(pred[0, 0])
    # Update the sequence: drop the first value and append the new prediction
    current_sequence = np.append(current_sequence[1:], [[pred[0, 0]]], axis=0)

# Inverse transform the scaled forecast values back to the original scale
forecast = scaler.inverse_transform(np.array(forecast_scaled).reshape(-1, 1))

# Create a date range for the forecast period (assuming monthly frequency)
last_date = trunc_input['ds'].iloc[-1]
forecast_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=12, freq='MS')

# Create a DataFrame for the forecast
forecast_df = pd.DataFrame({
    'ds': forecast_dates,
    'yhat': forecast.flatten()
})

# Merge forecast with original data for plotting
merged_df = pd.merge(trunc_input, forecast_df, on='ds', how='outer')
merged_df['combined'] = merged_df['y'].fillna(merged_df['yhat'])
merged_df['is_forecast'] = merged_df['y'].isna()

# Plot the results
plt.figure(figsize=(12,6))
sns.lineplot(data=merged_df, x='ds', y='combined', hue='is_forecast', palette=['blue', 'red'])
plt.title("LSTM Forecast")
plt.xlabel("Date")
plt.ylabel("Value")
plt.show()

# Optionally, print the tail to inspect the forecasted values
print(merged_df.tail(13))
